In [1]:
import pandas as pd
import numpy as np

In [2]:
from utils import *

In [3]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1LDKHVqSIoBeICC7VTxj1hmOcin67YgIG2h3WPbD1PAo/gviz/tq?tqx=out:csv&sheet=entreprises'

In [4]:
entreprises = (
    pd
    .read_csv(
        sheet_url,
        dtype={'siren': str}
    )
    .assign(
        siren = lambda df: df.siren.str.pad(9, 'left', '0')
    )
)

# entreprises

In [5]:
comptes = (
    pd
    .read_csv('data/ratios_inpi_bce.csv', sep=";",dtype={'siren': str})
    .assign(
        siren = lambda df: df.siren.str.pad(9, 'left', '0')
    )
    .query('type_bilan == "C"')
    .assign(
        date_cloture_exercice = lambda df: pd.to_datetime(df.date_cloture_exercice),
        cloture_exercice_annee = lambda df: pd.to_datetime(df.date_cloture_exercice).dt.to_period('y')
    )
)

# comptes

In [6]:
ca = (
    entreprises
    .query('~siren.isna()')
    .set_index('siren')
    [['nom']]
    .pipe(lambda df: (
        df
        .join(
            df
            .join(comptes.set_index('siren'))
            .pivot_table(
                index = 'siren',
                columns = 'cloture_exercice_annee',
                values = 'chiffre_d_affaires',
                aggfunc= 'last'
            )
        )
    ))
)

(
    ca
    .pipe(format_table)
)

,nom,2016,2017,2018,2019,2020,2021,2022
siren,,,,,,,,
352730394,Bolloré participations SE,nan,3 418 000,3 417 000,3 166 000,2 953 000,2 953 000,nan
055804124,Bolloré,151 643 000,nan,153 281,144 209 000,143 249 000,149 516 000,nan
797476256,Bolloré Transport & Logistics,nan,57 496 176,58 994 555,60 077 584,55 610 020,57 666 536,nan
421090051,Blue Solutions,111 501 000,80 089 000,36 002 000,26 586 000,39 415 103,80 022 696,nan
343134763,Vivendi,46 000 000,66 500 000,683 000 000,73 500 000,nan,nan,nan
335480265,Havas,nan,97 143 000,94 920 000,96 223 000,84 850 000,88 965 000,nan
349208447,Havas worldwide,nan,103 914 672,nan,106 543 622,86 811 584,96 273 369,nan
350314639,INTERCORPORATE (EURO RSCG EVENTS FRANCE),nan,3 489 066,nan,1 950 224,574 955,1 016 617,nan
403201767,Havas Media,nan,198 601 513,177 547 128,174 305 290,139 843 613,166 312 940,nan


In [7]:
(
    ca
    .select_dtypes(include=np.number)
    .sum()
    .to_frame()
    .T
    .pipe(format_table)
)

,2016,2017,2018,2019,2020,2021,2022
0,339 501 368,9 661 071 466,10 267 549 884,10 034 683 587,9 593 822 500,10 627 168 163,3 349 000 000


In [8]:
rn = (
    entreprises
    .query('~siren.isna()')
    .set_index('siren')
    [['nom']]
    .pipe(lambda df: (
        df
        .join(
            df
            .join(comptes.set_index('siren'))
            .pivot_table(
                index = 'siren',
                columns = 'cloture_exercice_annee',
                values = 'resultat_net',
                aggfunc= 'last'
            )
        )
    ))
)

rn.pipe(format_table)

,nom,2016,2017,2018,2019,2020,2021,2022
siren,,,,,,,,
352730394,Bolloré participations SE,nan,1 527 000,1 883 000,2 225 000,61 757 000,2 212 000,nan
055804124,Bolloré,43 252 000,nan,-135 150,16 744 000,181 084 000,235 740 000,nan
797476256,Bolloré Transport & Logistics,nan,-1 007 126,402 193,138 140,206 009,114 700,nan
421090051,Blue Solutions,7 897 000,-3 615 000,-12 033 000,-16 209 000,-39 088 100,-31 828 474,nan
343134763,Vivendi,1 609 500 000,703 100 000,951 300 000,1 729 800 000,nan,nan,nan
335480265,Havas,nan,79 727 000,6 328 000,20 263 000,-90 016 000,105 522 000,nan
349208447,Havas worldwide,nan,2 750 591,nan,-4 007 036,-7 575 176,2 480 853,nan
350314639,INTERCORPORATE (EURO RSCG EVENTS FRANCE),nan,4 665,nan,884,6 471,401,nan
403201767,Havas Media,nan,-5 334 634,-15 036 916,-13 406 981,-22 270 501,-10 501 399,nan


In [9]:
(
    rn
    .select_dtypes(include=np.number)
    .sum()
    .to_frame()
    .T
    .pipe(format_table)
)

,2016,2017,2018,2019,2020,2021,2022
0,1 659 776 497,779 742 462,944 982 712,2 011 401 729,103 094 804,511 325 067,-68 000 000
